<a href="https://colab.research.google.com/github/Xuehui-Jiang/Uni-Mannheim-Master-Thesis/blob/main/Experiment_Cities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cloning the pyRDF2Vec repo
!git clone https://github.com/IBCNServices/pyRDF2Vec.git

Cloning into 'pyRDF2Vec'...
remote: Enumerating objects: 7396, done.
remote: Counting objects: 100% (364/364), done.
remote: Compressing objects: 100% (220/220), done.
remote: Total 7396 (delta 221), reused 261 (delta 137), pack-reused 7032
Receiving objects: 100% (7396/7396), 5.32 MiB | 11.29 MiB/s, done.
Resolving deltas: 100% (5022/5022), done.


In [ ]:
!pip uninstall community
!pip install python-louvain
!pip install pyRDF2Vec --upgrade

Found existing installation: community 1.0.0b1
Uninstalling community-1.0.0b1:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/community-1.0.0b1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/community/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.10/dist-packages/community/community_louvain.py
    /usr/local/lib/python3.10/dist-packages/community/community_status.py
Proceed (Y/n)? Y
  Successfully uninstalled community-1.0.0b1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.6/204.6 kB 19.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install rdflib
!pip install networkx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.1 MB/s eta 0:00:00


In [ ]:
import random
import warnings
warnings.filterwarnings('ignore')
import functools
import itertools
from typing import List, Sequence, Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rdflib
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC

import sys
sys.path.append('pyRDF2Vec')
sys.path.append('pyRDF2Vec/pyrdf2vec')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from pyrdf2vec import RDF2VecTransformer
from pyrdf2vec.graphs import KG
from pyrdf2vec.embedders import Word2Vec
from pyrdf2vec.walkers import RandomWalker

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))


Saving CompleteDataset.tsv to CompleteDataset.tsv
User uploaded file "CompleteDataset.tsv" with length 20612 bytes


In [ ]:
# load training and testing entities
complete_df = pd.read_csv('CompleteDataset.tsv', sep='\t')

cities = complete_df['city_name'].tolist()
entities = complete_df['DBpedia_URL'].tolist()
labels = complete_df['label'].tolist()
reg_targets = complete_df['rating'].tolist()

In order to obtain the DBpedia knowledge graph related to the entities in the dataset, the first method is to use the SPARQL endpoint provided by DBpedia to execute the query, retrieving direct types from the ontology, as well as outgoing and ingoing edges.

In [ ]:
import math
import requests

In [ ]:
# DBpedia SPARQL endpoint base URL
endpoint_url = "https://dbpedia.org/sparql"

# Split entities into smaller chunks for batch processing
batch_size = 71
num_batches = math.ceil(len(entities) / batch_size)

all_results = []

# Iterate over batches
for batch_idx in range(num_batches):
    start_idx = batch_idx * batch_size
    end_idx = min((batch_idx + 1) * batch_size, len(entities))
    batch_entities = entities[start_idx:end_idx]

    # Create a formatted string with these entities for use in the SPARQL query
    batch_entities_str = ', '.join(f"dbr:{entity.split('/')[-1]}" for entity in batch_entities)

    # Update the SPARQL query to use the batch_entities_str
    sparql_query_batch = f"""
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX dbr: <http://dbpedia.org/resource/>

    CONSTRUCT {{?x rdf:type ?tx . }} WHERE {{
        {{?x rdf:type ?tx . FILTER (?x IN ({batch_entities_str})) FILTER (REGEX(?tx, "http://dbpedia.org/ontology/")) }}
    }}
    """

    # Execute the SPARQL query for the batch
    params["query"] = sparql_query_batch
    response = requests.get(endpoint_url, params=params)
    response.raise_for_status()
    all_results.append(response.text)

# Save all the results to an output.nt file
with open("direct_types_cities.nt", 'w') as f:
    for result in all_results:
        f.write(result)

print("All results saved to direct_types_cities.nt!")


All results saved to direct_types_cities.nt!


In [ ]:
# DBpedia SPARQL endpoint base URL
endpoint_url = "https://dbpedia.org/sparql"

# Split entities into smaller chunks for batch processing
batch_size = 71
num_batches = math.ceil(len(entities) / batch_size)

all_results = []

# Iterate over batches
for batch_idx in range(num_batches):
    start_idx = batch_idx * batch_size
    end_idx = min((batch_idx + 1) * batch_size, len(entities))
    batch_entities = entities[start_idx:end_idx]

    # Create a formatted string with these entities for use in the SPARQL query
    batch_entities_str = ', '.join(f"dbr:{entity.split('/')[-1]}" for entity in batch_entities)

    # Update the SPARQL query to use the batch_entities_str
    sparql_query_batch = f"""
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX dbr: <http://dbpedia.org/resource/>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>

    CONSTRUCT {{
        ?x ?y ?z .
        ?z rdf:type ?tz .
    }}
    WHERE {{
        {{ ?x ?y ?z .
        OPTIONAL {{ ?z rdf:type ?tz . FILTER (REGEX(?tz, "http://dbpedia.org/ontology/")) }}
        ?y a owl:ObjectProperty .
        FILTER (?x IN ({batch_entities_str}))
        FILTER (!REGEX(?y, "http://dbpedia.org/ontology/wiki"))
        FILTER (REGEX(?z, "http://dbpedia.org/resource/")) }}
    }}
    """

    # Execute the SPARQL query for the batch
    params["query"] = sparql_query_batch
    response = requests.get(endpoint_url, params=params)
    response.raise_for_status()
    all_results.append(response.text)

# Save all the results to an output.nt file
with open("outgoing_edges_cities.nt", 'w') as f:
    for result in all_results:
        f.write(result)

print("All results saved to outgoing_edges_cities.nt!")


All results saved to outgoing_edges_cities.nt!


In [ ]:
import time

In [ ]:
# DBpedia SPARQL endpoint base URL
endpoint_url = "https://dbpedia.org/sparql"

# Split entities into smaller chunks for batch processing
batch_size = 5
num_batches = math.ceil(len(entities) / batch_size)

all_results = []
failed_batches = []

max_retries = 3

# Iterate over batches
for batch_idx in range(num_batches):
    start_idx = batch_idx * batch_size
    end_idx = min((batch_idx + 1) * batch_size, len(entities))
    batch_entities = entities[start_idx:end_idx]

    # Create a formatted string with these entities for use in the SPARQL query
    batch_entities_str = ', '.join(f"dbr:{entity.split('/')[-1]}" for entity in batch_entities)

    # Update the SPARQL query to use the batch_entities_str
    sparql_query_batch = f"""
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX dbr: <http://dbpedia.org/resource/>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>

    CONSTRUCT {{
        ?a ?r ?x .
        ?a rdf:type ?ta .
    }}
    WHERE {{
        {{ ?a ?r ?x .
        ?r a owl:ObjectProperty .
        FILTER (?x IN ({batch_entities_str})) }}
        FILTER (!REGEX(?r, "http://dbpedia.org/ontology/wiki"))
        FILTER (REGEX(?a, "http://dbpedia.org/resource/"))
    }}
    """

    retries = 0
    success = False
    while retries < max_retries and not success:
        try:
            # Execute the SPARQL query for the batch
            params["query"] = sparql_query_batch
            response = requests.get(endpoint_url, params=params)
            response.raise_for_status()
            all_results.append(response.text)
            success = True
        except:
            retries += 1
            time.sleep(2)  # Wait for 2 seconds before retrying

    # If all retries failed, record the batch_idx to failed_batches
    if not success:
        failed_batches.append(batch_idx)

# Save all the results to an output.nt file
with open("ingoing_edges_cities.nt", 'w') as f:
    for result in all_results:
        f.write(result)

print("All results saved to ingoing_edges_cities.nt!")
if failed_batches:
    print(f"Failed batches: {failed_batches}")


All results saved to ingoing_edges_cities.nt!
Failed batches: [0, 1, 3, 4, 5, 6, 7, 8, 9, 12, 13, 14, 27, 31, 32, 40]


In [ ]:
for batch_idx in failed_batches:
    start_idx = batch_idx * batch_size
    end_idx = min((batch_idx + 1) * batch_size, len(entities))
    failed_batch_entities = entities[start_idx:end_idx]

    for entity in failed_batch_entities:
        batch_entities_str = f"dbr:{entity.split('/')[-1]}"

        sparql_query_single = f"""
        PREFIX dbo: <http://dbpedia.org/ontology/>
        PREFIX dbr: <http://dbpedia.org/resource/>
        PREFIX owl: <http://www.w3.org/2002/07/owl#>

        CONSTRUCT {{
            ?a ?r ?x .
            ?a rdf:type ?ta .
        }}
        WHERE {{
            {{ ?a ?r ?x .
            ?r a owl:ObjectProperty .
            FILTER (?x IN ({batch_entities_str})) }}
            FILTER (!REGEX(?r, "http://dbpedia.org/ontology/wiki"))
            FILTER (REGEX(?a, "http://dbpedia.org/resource/"))
        }}
        """

        try:
            params["query"] = sparql_query_single
            response = requests.get(endpoint_url, params=params)
            response.raise_for_status()
            all_results.append(response.text)
        except:
            print(f"Failed to query for individual entity: {entity}")

# Save the updated results to an output.nt file
with open("extra_ingoing_edges_cities.nt", 'w') as f:
    for result in all_results:
        f.write(result)

print("All updated results saved to extra_ingoing_edges_cities.nt!")


Failed to query for individual entity: http://dbpedia.org/resource/Vienna
Failed to query for individual entity: http://dbpedia.org/resource/Brussels
Failed to query for individual entity: http://dbpedia.org/resource/Paris
Failed to query for individual entity: http://dbpedia.org/resource/Montreal
Failed to query for individual entity: http://dbpedia.org/resource/Wellington
Failed to query for individual entity: http://dbpedia.org/resource/London
Failed to query for individual entity: http://dbpedia.org/resource/Kobe
Failed to query for individual entity: http://dbpedia.org/resource/Dublin
Failed to query for individual entity: http://dbpedia.org/resource/Los_Angeles
Failed to query for individual entity: http://dbpedia.org/resource/Singapore
Failed to query for individual entity: http://dbpedia.org/resource/Detroit
Failed to query for individual entity: http://dbpedia.org/resource/Buenos_Aires
All updated results saved to extra_ingoing_edges_cities.nt!


In [ ]:
failed_entities = [
    "http://dbpedia.org/resource/Vienna",
    "http://dbpedia.org/resource/Brussels",
    "http://dbpedia.org/resource/Paris",
    "http://dbpedia.org/resource/Montreal",
    "http://dbpedia.org/resource/Wellington",
    "http://dbpedia.org/resource/London",
    "http://dbpedia.org/resource/Kobe",
    "http://dbpedia.org/resource/Dublin",
    "http://dbpedia.org/resource/Los_Angeles",
    "http://dbpedia.org/resource/Singapore",
    "http://dbpedia.org/resource/Detroit",
    "http://dbpedia.org/resource/Buenos_Aires"
]

individual_results = []

for entity in failed_entities:
    entity_str = f"dbr:{entity.split('/')[-1]}"

    sparql_query_single = f"""
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX dbr: <http://dbpedia.org/resource/>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>

    CONSTRUCT {{
        ?a ?r ?x .
        ?a rdf:type ?ta .
    }}
    WHERE {{
        {{ ?a ?r ?x .
        ?r a owl:ObjectProperty .
        FILTER (?x IN ({entity_str})) }}
        FILTER (!REGEX(?r, "http://dbpedia.org/ontology/wiki"))
        FILTER (REGEX(?a, "http://dbpedia.org/resource/"))
    }}
    """

    try:
        params["query"] = sparql_query_single
        response = requests.get(endpoint_url, params=params)
        response.raise_for_status()
        individual_results.append(response.text)
        print(f"Successfully queried for entity: {entity}")
        time.sleep(20)  # Wait for 20 seconds before the next query
    except:
        print(f"Failed to query for individual entity: {entity}")

# Save the individual results to an output.nt file
with open("individual_ingoing_edges_cities.nt", 'w') as f:
    for result in individual_results:
        f.write(result)

print("All individual results saved to individual_ingoing_edges_cities.nt!")


Successfully queried for entity: http://dbpedia.org/resource/Vienna
Successfully queried for entity: http://dbpedia.org/resource/Brussels
Failed to query for individual entity: http://dbpedia.org/resource/Paris
Successfully queried for entity: http://dbpedia.org/resource/Montreal
Successfully queried for entity: http://dbpedia.org/resource/Wellington
Failed to query for individual entity: http://dbpedia.org/resource/London
Successfully queried for entity: http://dbpedia.org/resource/Kobe
Successfully queried for entity: http://dbpedia.org/resource/Dublin
Failed to query for individual entity: http://dbpedia.org/resource/Los_Angeles
Successfully queried for entity: http://dbpedia.org/resource/Singapore
Failed to query for individual entity: http://dbpedia.org/resource/Detroit
Successfully queried for entity: http://dbpedia.org/resource/Buenos_Aires
All individual results saved to individual_ingoing_edges_cities.nt!


In [ ]:
# List of entities that failed in the last run
failed_again_entities = [
    "http://dbpedia.org/resource/Paris",
    "http://dbpedia.org/resource/London",
    "http://dbpedia.org/resource/Los_Angeles",
    "http://dbpedia.org/resource/Detroit"
]

individual_results_again = []

for entity in failed_again_entities:
    entity_str = f"dbr:{entity.split('/')[-1]}"

    sparql_query_single = f"""
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX dbr: <http://dbpedia.org/resource/>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>

    CONSTRUCT {{
        ?a ?r ?x .
        ?a rdf:type ?ta .
    }}
    WHERE {{
        {{ ?a ?r ?x .
        ?r a owl:ObjectProperty .
        FILTER (?x IN ({entity_str})) }}
        FILTER (!REGEX(?r, "http://dbpedia.org/ontology/wiki"))
        FILTER (REGEX(?a, "http://dbpedia.org/resource/"))
    }}
    """

    try:
        params["query"] = sparql_query_single
        response = requests.get(endpoint_url, params=params)
        response.raise_for_status()
        individual_results_again.append(response.text)
        print(f"Successfully queried for entity: {entity}")
        time.sleep(40)  # Increased wait time to 40 seconds before the next query
    except:
        print(f"Failed to query for individual entity: {entity}")

# Append the new individual results to the previous file
with open("individual_ingoing_edges_cities.nt", 'a') as f:
    for result in individual_results_again:
        f.write(result)

print("All additional individual results appended to individual_ingoing_edges_cities.nt!")


Failed to query for individual entity: http://dbpedia.org/resource/Paris
Failed to query for individual entity: http://dbpedia.org/resource/London
Successfully queried for entity: http://dbpedia.org/resource/Los_Angeles
Failed to query for individual entity: http://dbpedia.org/resource/Detroit
All additional individual results appended to individual_ingoing_edges_cities.nt!


In [ ]:
# Entities will not used in the experiments
absent_entities = [
    "http://dbpedia.org/resource/Paris",
    "http://dbpedia.org/resource/London",
    "http://dbpedia.org/resource/Detroit"
]

# Filter out absent entities
filtered_entities = [e for e in entities if e not in absent_entities]
filtered_labels = [label for entity, label in zip(entities, labels) if entity not in absent_entities]
filtered_reg_targets = [reg_targets for entity, reg_targets in zip(entities, reg_targets) if entity not in absent_entities]

In [ ]:
filenames = ["direct_types_cities.nt", "outgoing_edges_cities.nt", "ingoing_edges_cities.nt", "extra_ingoing_edges_cities.nt", "individual_ingoing_edges_cities.nt"]
with open("cities.nt", 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)

print("All files have been merged into combined_graph.nt!")


All files have been merged into combined_graph.nt!


In [ ]:
import shutil

destination_path = "/content/drive/My Drive/dbpedia_files"

shutil.copy("cities.nt", destination_path)

print(f"File saved to {destination_path}")

File saved to /content/drive/My Drive/dbpedia_files


The second method, which is the method we finally adopted to obtain the knowledge graph, is to download important files directly. And then Read each file and retain only those triples that are neither datatype properties nor literals.

In [ ]:
# Define the list of files
files_to_merge = [
    "instance_types_en.ttl",
    "infobox_properties_en.ttl",
    "article_categories_en.ttl",
    "disambiguations_en.ttl"
]

# Merge files
with open("direct_merged_dbpedia.ttl", "w") as outfile:
    for file in files_to_merge:
        with open("/content/drive/My Drive/dbpedia_files/" + file, "r") as infile:
            for line in infile:
                outfile.write(line)

print("Files have been merged into merged.ttl")


Files have been merged into merged.ttl


In [ ]:
# Define the list of files
files_to_merge = [
    "mappingbased_objects_en.ttl",
    "geonames_links_en.ttl"
]

# Merge files
with open("indirect_merged_dbpedia.ttl", "w") as outfile:
    for file in files_to_merge:
        with open("/content/drive/My Drive/dbpedia_files/" + file, "r") as infile:
            for line in infile:
                outfile.write(line)

print("Files have been merged into merged.ttl")


Files have been merged into merged.ttl


For files that describe core relationships and entity types, we choose to filter directly for triples related to entities in the Dataset.
For files that may provide more context and indirect relationships, we take a recursive approach to get entities and relationships that are directly or indirectly related to the target entity。

In [ ]:
# Filter out entities and relationships directly related to Cities Dataset
with open("direct_merged_dbpedia.ttl", "r") as file, open("direct_cities_filtered_dbpedia.ttl", "w") as out_file:
    for line in file:
        triple = line.strip().split()
        if len(triple) > 2:
            # Check if the subject or object of the triple is in our city URL list
            if triple[0].strip('<>') in entities or triple[2].strip('<>') in entities:
                out_file.write(line)

In [ ]:
# Filter out entities and relationships directly or indirectly related to Cities Dataset
entities_related_to_cities = set(entities)
new_entities = set(entities)
iteration_count = 0

while new_entities and iteration_count < 2:
    temp_entities = set()
    with open("indirect_merged_dbpedia.ttl", "r") as file:
        for line in file:
            triple = line.strip().split()
            if len(triple) > 2 and (triple[0].strip('<>') in new_entities or triple[2].strip('<>') in new_entities):
                    temp_entities.add(triple[0].strip('<>'))
                    temp_entities.add(triple[2].strip('<>'))

    new_entities = temp_entities - entities_related_to_cities
    entities_related_to_cities.update(new_entities)
    iteration_count += 1

# Finally, we can write all triples that are directly or indirectly related to entities in the `entities_related_to_cities` collection
with open("indirect_merged_dbpedia.ttl", "r") as file, open("indirect_cities_filtered_dbpedia.ttl", "w") as out_file:
    for line in file:
        triple = line.strip().split()
        if len(triple) > 2 and (triple[0].strip('<>') in entities_related_to_cities or triple[2].strip('<>') in entities_related_to_cities):
            out_file.write(line)


In [ ]:
# Define the list of files
files_to_merge = [
    "direct_cities_filtered_dbpedia.ttl",
    "indirect_cities_filtered_dbpedia.ttl"
]

# Merge files
with open("cities_filtered_dbpedia.ttl", "w") as outfile:
    for file in files_to_merge:
        with open(file, "r") as infile:
            for line in infile:
                outfile.write(line)

print("Files have been merged into merged.ttl")


Files have been merged into merged.ttl


In [ ]:
input_file = "cities_filtered_dbpedia.ttl"
output_file = "cleaned_cities_filtered_dbpedia.ttl"

with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    for line in infile:
        # Determine whether it is a comment or a blank line
        if line.startswith("#") or line.strip() == "":
            continue

        # Split triples
        triple = line.strip().split()

        # Make sure this is a complete triplet
        if len(triple) > 2:
            # Check whether the object starts with "http://dbpedia.org/resource/"
            if triple[2].startswith("<http://dbpedia.org/resource/"):
                outfile.write(line)

print(f"Finished filtering {input_file} and saved the result to {output_file}")


Finished filtering cities_filtered_dbpedia.ttl and saved the result to cleaned_cities_filtered_dbpedia.ttl


In [ ]:
!head -n 10 cleaned_cities_filtered_dbpedia.ttl

<http://dbpedia.org/resource/Alberta> <http://dbpedia.org/property/largestcity> <http://dbpedia.org/resource/Calgary> .
<http://dbpedia.org/resource/Azerbaijan> <http://dbpedia.org/property/largestCities> <http://dbpedia.org/resource/Baku> .
<http://dbpedia.org/resource/Amsterdam> <http://dbpedia.org/property/subdivisionType> <http://dbpedia.org/resource/Provinces_of_the_Netherlands> .
<http://dbpedia.org/resource/Amsterdam> <http://dbpedia.org/property/areaBlank1Title> <http://dbpedia.org/resource/Randstad> .
<http://dbpedia.org/resource/Amsterdam> <http://dbpedia.org/property/populationBlank1Title> <http://dbpedia.org/resource/Amsterdam_Metropolitan_Area> .
<http://dbpedia.org/resource/Amsterdam> <http://dbpedia.org/property/populationBlank2Title> <http://dbpedia.org/resource/Randstad> .
<http://dbpedia.org/resource/Amsterdam> <http://dbpedia.org/property/postalCodeType> <http://dbpedia.org/resource/Postal_codes_in_the_Netherlands> .
<http://dbpedia.org/resource/Amsterdam> <http://db

In [ ]:
import shutil

destination_path = "/content/drive/My Drive/dbpedia_files"

shutil.copy("cleaned_cities_filtered_dbpedia.ttl", destination_path)

print(f"File saved to {destination_path}")

File saved to /content/drive/My Drive/dbpedia_files


In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("/content/drive/My Drive/dbpedia_files/cleaned_cities_filtered_dbpedia.ttl", fmt='turtle')

kgentities = kg._entities

In [ ]:
for e in entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

http://dbpedia.org/resource/Winston_Salem
http://dbpedia.org/resource/Omuta
http://dbpedia.org/resource/Rio_De_Janeiro
http://dbpedia.org/resource/Port_Au_Prince


In [ ]:
# Entities not in KG
absent_entities = [
    "http://dbpedia.org/resource/Winston_Salem",
    "http://dbpedia.org/resource/Omuta",
    "http://dbpedia.org/resource/Rio_De_Janeiro",
    "http://dbpedia.org/resource/Port_Au_Prince"
    ]

# Filter out absent entities
filtered_entities = [e for e in entities if e not in absent_entities]
filtered_labels = [label for entity, label in zip(entities, labels) if entity not in absent_entities]
filtered_reg_targets = [reg_targets for entity, reg_targets in zip(entities, reg_targets) if entity not in absent_entities]

In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

Prepare for Knowledge Graph

In [ ]:
from rdflib import Graph, URIRef, Literal, Namespace
import networkx as nx

In [ ]:
# Load the graph from the input file
graph = Graph()
graph.parse("/content/drive/My Drive/dbpedia_files/cleaned_cities_filtered_dbpedia.ttl", format="turtle")

<Graph identifier=N09f515e9152442eaa5abc12f75ff9dfb (<class 'rdflib.graph.Graph'>)>

In [ ]:
G = nx.DiGraph()  # Create a directed graph

for s, p, o in graph:
    G.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G.nodes())
num_edges = len(G.edges())
print("The DBpedia knowledge graph as of 2016-10 has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The DBpedia knowledge graph as of 2016-10 has 4763333 nodes and 14631598 edges.



Classification & Regression tasks based graph embeddings without SA

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV, KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import SVC, SVR
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer, mean_squared_error
import pandas as pd

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('ttl_class_reg_results.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.787143
1           cbow         50                 C4.5  Accuracy   0.735476
2           cbow         50            KNN (k=3)  Accuracy   0.792143
3           cbow         50                  SVM  Accuracy   0.845000
4           cbow         50    Linear Regression      RMSE  14.889069


In [ ]:
from google.colab import files
files.download('pure_class_reg_results.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Classification & Regression tasks based graph embeddings extracted on subgraph base on SA with BFS

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving main.py to main.py
Saving sa_helper.py to sa_helper.py


In [ ]:
from main import *
from sa_helper import *

In [ ]:
# Count the neighbor node number for each nodes in the graph
neighbor_counts = {node: len(list(G.neighbors(node))) for node in G.nodes()}

strict=True, excl=False, pop=False

In [ ]:
subgraph_F_F_s = spreading_activation_BFS(graph, G, filtered_entities, neighbor_counts,
                                    alpha=0.186, beta=0.006, max_hops=3, extraction_threshold=0,
                                    strict=True, fan_out=True, excl=False, pop=False)

In [ ]:
G_sub_s = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_F_F_s:
    G_sub_s.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub_s.nodes())
num_edges = len(G_sub_s.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 946 nodes and 2431 edges.


In [ ]:
# Serialize the graph to TTL format
nt_data = subgraph_F_F_s.serialize(format='nt')

# Save to a TTL file
with open('BFS_subgraph_s.nt', 'w') as output_file:
    output_file.write(nt_data)

In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("BFS_subgraph_s.nt", fmt='nt')

kgentities = kg._entities

In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

http://dbpedia.org/resource/Zurich
http://dbpedia.org/resource/Dusseldorf
http://dbpedia.org/resource/Nurnberg
http://dbpedia.org/resource/Washington_Dc
http://dbpedia.org/resource/Tsukuba
http://dbpedia.org/resource/St._Peter_Port
http://dbpedia.org/resource/Yokkaichi
http://dbpedia.org/resource/Katsuyama
http://dbpedia.org/resource/San_Jose
http://dbpedia.org/resource/Brasilia
http://dbpedia.org/resource/Noumea
http://dbpedia.org/resource/Johor_Baharu
http://dbpedia.org/resource/Sao_Paulo
http://dbpedia.org/resource/Asuncion
http://dbpedia.org/resource/Bogota
http://dbpedia.org/resource/Blantyre
http://dbpedia.org/resource/Medellin
http://dbpedia.org/resource/St._Petersburg
http://dbpedia.org/resource/Madras
http://dbpedia.org/resource/Yaounde
http://dbpedia.org/resource/Dacca
http://dbpedia.org/resource/Dar_Es_Salaam
http://dbpedia.org/resource/Ndjamena
http://dbpedia.org/resource/Pointe_Noire


In [ ]:
# Entities not in KG
absent_entities = [
    "http://dbpedia.org/resource/Zurich",
    "http://dbpedia.org/resource/Dusseldorf",
    "http://dbpedia.org/resource/Nurnberg",
    "http://dbpedia.org/resource/Washington_Dc",
    "http://dbpedia.org/resource/Tsukuba",
    "http://dbpedia.org/resource/St._Peter_Port",
    "http://dbpedia.org/resource/Yokkaichi",
    "http://dbpedia.org/resource/Katsuyama",
    "http://dbpedia.org/resource/San_Jose",
    "http://dbpedia.org/resource/Brasilia",
    "http://dbpedia.org/resource/Noumea",
    "http://dbpedia.org/resource/Johor_Baharu",
    "http://dbpedia.org/resource/Sao_Paulo",
    "http://dbpedia.org/resource/Asuncion",
    "http://dbpedia.org/resource/Bogota",
    "http://dbpedia.org/resource/Blantyre",
    "http://dbpedia.org/resource/Medellin",
    "http://dbpedia.org/resource/St._Petersburg",
    "http://dbpedia.org/resource/Madras",
    "http://dbpedia.org/resource/Yaounde",
    "http://dbpedia.org/resource/Dacca",
    "http://dbpedia.org/resource/Dar_Es_Salaam",
    "http://dbpedia.org/resource/Ndjamena",
    "http://dbpedia.org/resource/Pointe_Noire"
]

# Filter out absent entities
fil_filtered_entities = [e for e in filtered_entities if e not in absent_entities]
fil_filtered_labels = [label for entity, label in zip(filtered_entities, labels) if entity not in absent_entities]
fil_filtered_reg_targets = [reg_targets for entity, reg_targets in zip(filtered_entities, reg_targets) if entity not in absent_entities]

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500, with_reverse=False)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, fil_filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, fil_filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, fil_filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('BFS_ttl_class_reg_results_s.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.411988
1           cbow         50                 C4.5  Accuracy   0.544152
2           cbow         50            KNN (k=3)  Accuracy   0.521345
3           cbow         50                  SVM  Accuracy   0.553509
4           cbow         50    Linear Regression      RMSE  28.118587


strict=False, excl=False, pop=False

In [ ]:
subgraph_F_F = spreading_activation_BFS(graph, G, filtered_entities, neighbor_counts,
                                    alpha=0.186, beta=0.006, max_hops=3, extraction_threshold=0,
                                    strict=False, fan_out=True, excl=False, pop=False)

In [ ]:
G_sub = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_F_F:
    G_sub.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub.nodes())
num_edges = len(G_sub.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 1151814 nodes and 1418106 edges.


In [ ]:
# Serialize the graph to TTL format
nt_data = subgraph_F_F.serialize(format='nt')

# Save to a TTL file
with open('BFS_subgraph_loose.nt', 'w') as output_file:
    output_file.write(nt_data)

In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("BFS_subgraph_loose.nt", fmt='nt')

kgentities = kg._entities

In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500, with_reverse=False)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('BFS_ttl_class_reg_results.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.782381
1           cbow         50                 C4.5  Accuracy   0.632381
2           cbow         50            KNN (k=3)  Accuracy   0.782143
3           cbow         50                  SVM  Accuracy   0.835714
4           cbow         50    Linear Regression      RMSE  16.260528


other schemes

excl=True, pop=False

In [ ]:
subgraph_T_F = spreading_activation_BFS(graph, G, filtered_entities, neighbor_counts,
                                    alpha=0.186, beta=0.006, max_hops=3, extraction_threshold=0,
                                    strict=False, fan_out=True, excl=True, pop=False)

In [ ]:
G_sub_tf = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_T_F:
    G_sub_tf.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub_tf.nodes())
num_edges = len(G_sub_tf.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 1155807 nodes and 1427512 edges.


In [ ]:
# Serialize the graph to TTL format
nt_data = subgraph_T_F.serialize(format='nt')

# Save to a TTL file
with open('BFS_subgraph_tf.nt', 'w') as output_file:
    output_file.write(nt_data)

In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("BFS_subgraph_tf.nt", fmt='nt')

kgentities = kg._entities

In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500, with_reverse=False)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('BFS_ttl_class_reg_results_tf.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.768333
1           cbow         50                 C4.5  Accuracy   0.646429
2           cbow         50            KNN (k=3)  Accuracy   0.787143
3           cbow         50                  SVM  Accuracy   0.845476
4           cbow         50    Linear Regression      RMSE  16.226600


excl=False, pop=True

In [ ]:
subgraph_F_T = spreading_activation_BFS(graph, G, filtered_entities, neighbor_counts, alpha=0.186,
                                        beta=0.006, max_hops=3, extraction_threshold=0,
                                        strict=False, fan_out=True, excl=False, pop=True)

In [ ]:
G_sub_ft = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_F_T:
    G_sub_ft.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub_ft.nodes())
num_edges = len(G_sub_ft.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 273098 nodes and 311897 edges.


In [ ]:
# Serialize the graph to TTL format
nt_data = subgraph_F_T.serialize(format='nt')

# Save to a TTL file
with open('BFS_subgraph_ft.nt', 'w') as output_file:
    output_file.write(nt_data)

In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("BFS_subgraph_ft.nt", fmt='nt')

kgentities = kg._entities

In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500, with_reverse=False)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('BFS_ttl_class_reg_results_ft.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.577143
1           cbow         50                 C4.5  Accuracy   0.628095
2           cbow         50            KNN (k=3)  Accuracy   0.723810
3           cbow         50                  SVM  Accuracy   0.767857
4           cbow         50    Linear Regression      RMSE  19.939133


excl=True, pop=True

In [ ]:
subgraph_T_T = spreading_activation_BFS(graph, G, filtered_entities, neighbor_counts, alpha=0.186,
                                        beta=0.006, max_hops=3, extraction_threshold=0,
                                        strict=False, fan_out=True, excl=True, pop=True)

In [ ]:
G_sub_tt = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_T_T:
    G_sub_tt.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub_tt.nodes())
num_edges = len(G_sub_tt.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 273098 nodes and 311897 edges.


In [ ]:
# Serialize the graph to TTL format
nt_data = subgraph_T_T.serialize(format='nt')

# Save to a TTL file
with open('BFS_subgraph_tt.nt', 'w') as output_file:
    output_file.write(nt_data)

In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("BFS_subgraph_tt.nt", fmt='nt')

kgentities = kg._entities

In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500, with_reverse=False)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('BFS_ttl_class_reg_results_tt.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.605476
1           cbow         50                 C4.5  Accuracy   0.700714
2           cbow         50            KNN (k=3)  Accuracy   0.714286
3           cbow         50                  SVM  Accuracy   0.791190
4           cbow         50    Linear Regression      RMSE  19.313904


Classification & Regression tasks based graph embeddings extracted on subgraph base on SA with DFS

excl=False, pop=False

In [ ]:
subgraph_F_F = spreading_activation_DFS(graph, G, filtered_entities, neighbor_counts, alpha=0.1,
                                        beta=0.009, max_depth=2, extraction_threshold=0,
                                        strict=False, fan_out=True, excl=False, pop=False)

In [ ]:
G_sub_ff = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_F_F:
    G_sub_ff.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub_ff.nodes())
num_edges = len(G_sub_ff.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 273097 nodes and 311897 edges.


In [ ]:
# Serialize the graph to TTL format
nt_data = subgraph_F_F.serialize(format='nt')

# Save to a TTL file
with open('DFS_subgraph_ff.nt', 'w') as output_file:
    output_file.write(nt_data)

In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("DFS_subgraph_ff.nt", fmt='nt')

kgentities = kg._entities

In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500, with_reverse=False)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('DFS_ttl_class_reg_results_ff.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.561667
1           cbow         50                 C4.5  Accuracy   0.604524
2           cbow         50            KNN (k=3)  Accuracy   0.715238
3           cbow         50                  SVM  Accuracy   0.762857
4           cbow         50    Linear Regression      RMSE  20.010073


excl=True, pop=False

In [ ]:
subgraph_T_F = spreading_activation_DFS(graph, G, filtered_entities, neighbor_counts,
                                    alpha=0.1, beta=0.009, max_depth=2, extraction_threshold=0,
                                    strict=False, fan_out=True, excl=True, pop=False)

In [ ]:
G_sub_tf = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_T_F:
    G_sub_tf.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub_tf.nodes())
num_edges = len(G_sub_tf.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 273097 nodes and 311897 edges.


In [ ]:
# Serialize the graph to TTL format
nt_data = subgraph_T_F.serialize(format='nt')

# Save to a TTL file
with open('DFS_subgraph_tf.nt', 'w') as output_file:
    output_file.write(nt_data)

In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("DFS_subgraph_tf.nt", fmt='nt')

kgentities = kg._entities

In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500, with_reverse=False)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('DFS_ttl_class_reg_results_tf.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.600714
1           cbow         50                 C4.5  Accuracy   0.613333
2           cbow         50            KNN (k=3)  Accuracy   0.700952
3           cbow         50                  SVM  Accuracy   0.734286
4           cbow         50    Linear Regression      RMSE  19.643796


excl=False, pop=True

In [ ]:
subgraph_F_T = spreading_activation_DFS(graph, G, filtered_entities, neighbor_counts, alpha=0.1,
                                        beta=0.009, max_depth=2, extraction_threshold=0,
                                        strict=False, fan_out=True, excl=False, pop=True)

In [ ]:
G_sub_ft = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_F_T:
    G_sub_ft.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub_ft.nodes())
num_edges = len(G_sub_ft.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 273097 nodes and 311897 edges.


In [ ]:
# Serialize the graph to TTL format
nt_data = subgraph_F_T.serialize(format='nt')

# Save to a TTL file
with open('DFS_subgraph_ft.nt', 'w') as output_file:
    output_file.write(nt_data)

In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("DFS_subgraph_ft.nt", fmt='nt')

kgentities = kg._entities

In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500, with_reverse=False)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('DFS_ttl_class_reg_results_ft.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.581667
1           cbow         50                 C4.5  Accuracy   0.667619
2           cbow         50            KNN (k=3)  Accuracy   0.733333
3           cbow         50                  SVM  Accuracy   0.753095
4           cbow         50    Linear Regression      RMSE  20.475899


excl=True, pop=True

In [ ]:
subgraph_T_T = spreading_activation_DFS(graph, G, filtered_entities, neighbor_counts, alpha=0.1,
                                        beta=0.009, max_depth=2, extraction_threshold=0,
                                        strict=False, fan_out=True, excl=True, pop=True)

In [ ]:
G_sub_tt = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_T_T:
    G_sub_tt.add_edge(s, o, predicate=p)

In [ ]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub_tt.nodes())
num_edges = len(G_sub_tt.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 273097 nodes and 311897 edges.


In [ ]:
# Serialize the graph to TTL format
nt_data = subgraph_T_T.serialize(format='nt')

# Save to a TTL file
with open('DFS_subgraph_tt.nt', 'w') as output_file:
    output_file.write(nt_data)

In [ ]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("DFS_subgraph_tt.nt", fmt='nt')

kgentities = kg._entities

In [ ]:
for e in filtered_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

In [ ]:
# Initialize the results dataframe
results = pd.DataFrame(columns=['Embedding Mode', 'Dimensions', 'Classifier/Regressor', 'Metric', 'Score'])

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500, with_reverse=False)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        # 2. Perform classification and regression tasks
        skf = StratifiedKFold(n_splits=10)
        kf = KFold(n_splits=10)

        # Classification
        classifiers = {
            'Naive Bayes': GaussianNB(),
            'C4.5': DecisionTreeClassifier(),
            'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
            'SVM': GridSearchCV(SVC(), param_grid={'C': [10**-3, 10**-2, 0.1, 1, 10, 10**2, 10**3]})
        }

        for name, clf in classifiers.items():
            scores = cross_val_score(clf, embeddings, filtered_labels, cv=skf, scoring='accuracy')
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'Accuracy', 'Score': scores.mean()}, ignore_index=True)

        # Regression
        regressors = {
            'Linear Regression': LinearRegression(),
            'KNN (k=3) Regressor': KNeighborsRegressor(n_neighbors=3),
            'M5Rules': DecisionTreeRegressor()
        }

        for name, reg in regressors.items():
            rmse_scorer = make_scorer(mean_squared_error, squared=False)
            scores = cross_val_score(reg, embeddings, filtered_reg_targets, cv=kf, scoring=rmse_scorer)
            results = results.append({'Embedding Mode': mode, 'Dimensions': dim, 'Classifier/Regressor': name, 'Metric': 'RMSE', 'Score': scores.mean()}, ignore_index=True)

# Save the results
results.to_csv('DFS_ttl_class_reg_results_tt.csv')

In [ ]:
print(results.head())

  Embedding Mode Dimensions Classifier/Regressor    Metric      Score
0           cbow         50          Naive Bayes  Accuracy   0.586905
1           cbow         50                 C4.5  Accuracy   0.584762
2           cbow         50            KNN (k=3)  Accuracy   0.699762
3           cbow         50                  SVM  Accuracy   0.767143
4           cbow         50    Linear Regression      RMSE  19.078100
